In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std


    return df_out


def summarize_feature_rr(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- RR summary ---
    if "RR" in d.columns:
        or_mean = d["RR"].mean()
        or_std = d["RR"].std(ddof=1)
        df_out.loc[feature, "RR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std


    return df_out

features = ["Mono", "One", "Two", "Three", "Total"] 




from scipy.stats import chi2
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import statsmodels.api as sm



def hosmer_lemeshow(y_true, y_prob, g=10, eps=1e-12):

    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()

    if hl_df.shape[0] < g:
        return np.nan, np.nan

    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")
    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    if hl_table.shape[0] < 3:
        return np.nan, np.nan

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]

    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"]) ** 2) / (hl_table["expected"] + eps) +
        ((hl_table["observed_neg"] - hl_table["expected_neg"]) ** 2) / (hl_table["expected_neg"] + eps)
    ).sum()

    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def binarize_with_thresholds(s, low_thr, high_thr):
    """
    > high_thr -> 1
    <= low_thr -> 0
    entre medio -> NaN
    """
    return np.where(
        s > high_thr, 1,
        np.where(s <= low_thr, 0, np.nan)
    )




def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Cross-sectional OR removing Slovakia

### Load data

In [3]:
import time

t0 = time.perf_counter()


In [4]:
data = pd.read_csv('data/BBAG-cross.csv')

In [5]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [6]:
data

,Unnamed: 0,record_id,country,year_acq_x,Sex_1F_2M,Education,Barthel,Diabetes_1Y_0N,Hypertension_1Y_0N,Heart_Disease_1Y_0N,...,number_stable_inst,distance,Proficiency,Educ_doct,Educ_upper_sec,Educ_master,Educ_bach,GAP_corrected,GAP_bin,y_pred_corrected
0,0,Share_AT-009869-01,Austria,2011,2,1,10.0,0.0,1.0,0.0,...,4,31,29.80,NaN,76.430000,NaN,NaN,-0.957336,0.0,62.042664
1,1,Share_AT-020669-02,Austria,2011,2,1,10.0,0.0,1.0,0.0,...,4,31,29.80,NaN,76.430000,NaN,NaN,-1.747516,0.0,60.252484
2,2,Share_AT-027123-02,Austria,2019,2,1,10.0,0.0,1.0,0.0,...,4,31,32.25,1.020000,80.669998,12.070000,15.36000,-1.349671,0.0,63.650329
3,3,Share_AT-028320-01,Austria,2019,2,2,10.0,0.0,1.0,0.0,...,4,31,32.25,1.020000,80.669998,12.070000,15.36000,-4.838502,0.0,70.161498
4,4,Share_AT-030094-01,Austria,2011,1,2,10.0,0.0,0.0,1.0,...,4,31,29.80,NaN,76.430000,NaN,NaN,2.180163,1.0,65.180163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84122,86142,Share_RO-722069-02,Romania,2019,1,2,10.0,0.0,0.0,0.0,...,2,144,18.55,0.105665,69.180000,11.291199,14.78602,-4.886865,0.0,60.113135
84123,86143,Share_SE-841359-05,Sweden,2019,1,2,9.0,0.0,0.0,0.0,...,0,21,55.35,1.290000,77.260002,11.850000,25.08000,2.974218,1.0,61.974218
84124,86144,Share_SI-234133-03,Slovenia,2019,1,2,10.0,0.0,0.0,0.0,...,1,12,47.15,3.580000,82.940002,16.240000,21.51000,-3.610002,0.0,50.389998
84125,86145,Share_SI-352598-02,Slovenia,2019,1,1,10.0,0.0,0.0,1.0,...,1,12,47.15,3.580000,82.940002,16.240000,21.51000,4.738801,1.0,63.738801


In [7]:
vars_ = ['Mono', 'One', 'Two', 'Three', 'Total']

results_merge_df_all = data.copy()

rows = []



for boosts in range(10): 
    for i in vars_:
        count = 0

        a = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 5)))
        b = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 25)))
        c = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 75)))
        d = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 95)))

        low_thresholds = range(a, b)
        high_thresholds = range(c, d)

        for low_thr in low_thresholds:
            for high_thr in high_thresholds:

                tmp = results_merge_df_all.copy()

                tmp[i] = binarize_with_thresholds(tmp[i], low_thr, high_thr)

                tmp = tmp.dropna(subset=[i, "GAP_bin"]).reset_index(drop=True)



                y_ols = tmp["GAP_bin"]
                X_ols = tmp[[i]]

                test_size = 500 / tmp.shape[0]


                X_train, X_test, y_train, y_test = train_test_split(
                    X_ols, y_ols,
                    test_size=test_size,
                    #stratify=tmp["GAP_bin"],
                    random_state=boosts
                )

                X_train = X_train.copy()
                X_test = X_test.copy()
                X_train["intercept"] = 1
                X_test["intercept"] = 1


                model = sm.Logit(y_train, X_train).fit(disp=0)


                params = model.params
                conf = np.exp(model.conf_int())
                conf["OR"] = np.exp(params)
                conf["z"] = model.tvalues
                conf["P>|z|"] = model.pvalues
                conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

                df = conf.loc[i:i].copy()
                df["Feature"] = i

                df["low_thr"] = low_thr
                df["high_thr"] = high_thr
                df["boosts"] = boosts
                df["n_tmp"] = tmp.shape[0]  


                rows.append(df)

                count += 1

df_directions_odd = pd.concat(rows, axis=0).reset_index(drop=True) if len(rows) else pd.DataFrame()

In [8]:
count*10

1100

In [9]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f)

df_directions_odd_f.to_excel('Results/cross/cross_OR_-removing-slovakia-binarization.xlsx')
df_directions_odd_f

,OR,2.5%,97.5%
Mono,2.174037,1.784902,2.563171
One,0.895277,0.845184,0.945370
Two,0.621144,0.550963,0.691325
Three,0.774117,0.645024,0.903209
Total,0.508102,0.456724,0.559481


In [10]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 5 min 56.6 s


## Cross-sectional OR with Slovakia

### Load data

In [11]:
import time

t0 = time.perf_counter()


In [12]:
data = pd.read_csv('data/BBAG-cross.csv')

In [13]:
data

,Unnamed: 0,record_id,country,year_acq_x,Sex_1F_2M,Education,Barthel,Diabetes_1Y_0N,Hypertension_1Y_0N,Heart_Disease_1Y_0N,...,number_stable_inst,distance,Proficiency,Educ_doct,Educ_upper_sec,Educ_master,Educ_bach,GAP_corrected,GAP_bin,y_pred_corrected
0,0,Share_AT-009869-01,Austria,2011,2,1,10.0,0.0,1.0,0.0,...,4,31,29.80,NaN,76.430000,NaN,NaN,-0.957336,0.0,62.042664
1,1,Share_AT-020669-02,Austria,2011,2,1,10.0,0.0,1.0,0.0,...,4,31,29.80,NaN,76.430000,NaN,NaN,-1.747516,0.0,60.252484
2,2,Share_AT-027123-02,Austria,2019,2,1,10.0,0.0,1.0,0.0,...,4,31,32.25,1.02,80.669998,12.07,15.360000,-1.349671,0.0,63.650329
3,3,Share_AT-028320-01,Austria,2019,2,2,10.0,0.0,1.0,0.0,...,4,31,32.25,1.02,80.669998,12.07,15.360000,-4.838502,0.0,70.161498
4,4,Share_AT-030094-01,Austria,2011,1,2,10.0,0.0,0.0,1.0,...,4,31,29.80,NaN,76.430000,NaN,NaN,2.180163,1.0,65.180163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86144,86144,Share_SI-234133-03,Slovenia,2019,1,2,10.0,0.0,0.0,0.0,...,1,12,47.15,3.58,82.940002,16.24,21.510000,-3.610002,0.0,50.389998
86145,86145,Share_SI-352598-02,Slovenia,2019,1,1,10.0,0.0,0.0,1.0,...,1,12,47.15,3.58,82.940002,16.24,21.510000,4.738801,1.0,63.738801
86146,86146,Share_SI-583115-02,Slovenia,2019,2,2,10.0,0.0,0.0,0.0,...,1,12,47.15,3.58,82.940002,16.24,21.510000,-6.653084,0.0,70.346916
86147,86147,Share_SK-112919-02,Slovakia,2019,1,2,9.0,0.0,0.0,0.0,...,4,67,38.45,0.84,87.699997,20.32,22.860001,0.850133,1.0,62.850133


In [14]:
vars_ = ['Mono', 'One', 'Two', 'Three', 'Total']

results_merge_df_all = data.copy()

rows = []

count = 0

for boosts in range(10): 
    for i in vars_:


        a = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 5)))
        b = int(np.floor(np.percentile(results_merge_df_all[i].dropna(), 25)))
        c = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 75)))
        d = int(np.ceil(np.percentile(results_merge_df_all[i].dropna(), 95)))

        low_thresholds = range(a, b)
        high_thresholds = range(c, d)

        for low_thr in low_thresholds:
            for high_thr in high_thresholds:

                tmp = results_merge_df_all.copy()

                tmp[i] = binarize_with_thresholds(tmp[i], low_thr, high_thr)

                tmp = tmp.dropna(subset=[i, "GAP_bin"]).reset_index(drop=True)



                y_ols = tmp["GAP_bin"]
                X_ols = tmp[[i]]

                test_size = 500 / tmp.shape[0]


                X_train, X_test, y_train, y_test = train_test_split(
                    X_ols, y_ols,
                    test_size=test_size,
                    #stratify=tmp["GAP_bin"],
                    random_state=boosts
                )

                X_train = X_train.copy()
                X_test = X_test.copy()
                X_train["intercept"] = 1
                X_test["intercept"] = 1


                model = sm.Logit(y_train, X_train).fit(disp=0)


                params = model.params
                conf = np.exp(model.conf_int())
                conf["OR"] = np.exp(params)
                conf["z"] = model.tvalues
                conf["P>|z|"] = model.pvalues
                conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

                df = conf.loc[i:i].copy()
                df["Feature"] = i

                df["low_thr"] = low_thr
                df["high_thr"] = high_thr
                df["boosts"] = boosts
                df["n_tmp"] = tmp.shape[0]  


                rows.append(df)

                count += 1

df_directions_odd = pd.concat(rows, axis=0).reset_index(drop=True) if len(rows) else pd.DataFrame()

In [15]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f)

df_directions_odd_f.to_excel('Results/cross/cross_OR_-with-slovakia-binarization.xlsx')
df_directions_odd_f

,OR,2.5%,97.5%
Mono,2.116086,1.735715,2.496457
One,0.895277,0.845184,0.945370
Two,0.608377,0.544658,0.672096
Three,0.772503,0.643513,0.901492
Total,0.516889,0.463897,0.569881


In [16]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 4 min 50.9 s
